<a href="https://colab.research.google.com/github/Cepera90/test/blob/main/test_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# Import libraries
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Open inc_points_c
TC = pd.read_csv("/content/drive/MyDrive/test/inc_points_c.csv")

In [ ]:
# See median values for drivers and passangers
TC.groupby('TrackOrigin').median()

,Unnamed: 0,IncomingTrackId,Speed,Latitude,Longitude,Height,Acceleration,Deceleration,TickTimestamp,AccelerationXOriginal,AccelerationYOriginal,AccelerationZOriginal
TrackOrigin,,,,,,,,,,,,
OriginalDriver,1383425.0,20681420.0,8.780098,53.214530,45.930267,107091.0,0.0,0.0,1.571309e+09,0.066023,0.162759,-0.040007
Passanger,3310451.0,20800891.0,31.359722,51.774608,55.232452,110196.0,0.0,0.0,1.571386e+09,0.065380,0.166199,-0.041804
Taxi,3233483.0,20713602.0,18.775861,55.717843,37.575511,155900.0,0.0,0.0,1.571312e+09,0.065457,0.159593,-0.020491


We see what median passanger user speed more when drivers speed (I think because professional drivers have strict speed limit)

In [ ]:
# Open rich_tracks_2
RT = pd.read_csv("/content/drive/MyDrive/test/rich_tracks_2.csv")

In [ ]:
# See standart deviation values for drivers and passangers
RT.groupby('TrackOrigin').std()

,Id,IncomingTrackId,Distance,Duration,AccelerationCount,DecelerationCount,Rating,PhoneUsage,OriginChanged,DistanceGPS,...,CorneringDuration,AccelerationICount,DecelerationICount,AccelerationIDuration,DecelerationIDuration,RatingCornering,VehicleId,BeaconId,DeviationRatio,DeviationMeters
TrackOrigin,,,,,,,,,,,,,,,,,,,,,
OriginalDriver,4801.492524,7.024230e+04,400.357699,28.220975,2.439093,3.154929,1.139604,2.468450,0.0,20.389829,...,1.515978,115.376788,220.519050,184.963118,297.982948,0.701323,NaN,NaN,0.000675,21.208906
Passanger,259711.240231,1.585408e+06,375.529387,38.344564,2.741259,3.029253,1.110365,7.581252,0.0,38.122624,...,9.396443,205.534600,339.757433,561.066446,435.932303,1.097489,NaN,NaN,0.000609,27.464519
Taxi,270605.859722,1.623501e+06,224.925853,29.744323,1.398211,2.161288,1.166092,2.183238,0.0,33.480420,...,1.296748,261.604315,195.238016,540.821350,295.079220,NaN,NaN,NaN,0.000000,26.437740


We see what standart deviation duration for passanger track more whan drivers track, so we can calculate duration from the TickTimestamp data in inc_point.

In [14]:
# Create new column Duration
TC['Duration'] = TC.groupby('IncomingTrackId').cumcount()

In [ ]:
#See variance values for drivers and passangers
RT.groupby('TrackOrigin').var()

,Id,IncomingTrackId,Distance,Duration,AccelerationCount,DecelerationCount,Rating,PhoneUsage,OriginChanged,DistanceGPS,...,CorneringDuration,AccelerationICount,DecelerationICount,AccelerationIDuration,DecelerationIDuration,RatingCornering,VehicleId,BeaconId,DeviationRatio,DeviationMeters
TrackOrigin,,,,,,,,,,,,,,,,,,,,,
OriginalDriver,2.305433e+07,4.933981e+09,160286.287334,796.423423,5.949172,9.953577,1.298698,6.093244,0.0,415.745136,...,2.298189,13311.803181,48628.651597,34211.355162,88793.837078,0.491854,NaN,NaN,4.556751e-07,449.817703
Passanger,6.744993e+10,2.513518e+12,141022.320872,1470.305551,7.514501,9.176376,1.232911,57.475385,0.0,1453.334452,...,88.293137,42244.471671,115435.113452,314795.556318,190036.972438,1.204483,NaN,NaN,3.712220e-07,754.299818
Taxi,7.322753e+10,2.635756e+12,50591.639241,884.724723,1.954993,4.671165,1.359771,4.766529,0.0,1120.938501,...,1.681556,68436.817409,38117.882897,292487.732281,87071.746019,NaN,NaN,NaN,0.000000e+00,698.954078


We see what variance of over speed mileage for passangers user more than drivers user, so we can add columns in inc_point data frame with over speed values.

In [17]:
#Delete the numeric data inc point, IncomingTrackId and track origin
TC = TC.drop(['Unnamed: 0','os_version','TickTimestamp','PointDate', 'IncomingTrackId'], axis=1)

In [18]:
#Delete the Latitude, Longitude and Height columns for anonimized data
TC = TC.drop(['Latitude','Longitude','Height'], axis=1)

In [19]:
# 'OriginalDriver' and 'Taxi' - 'Driver'
TC = TC.replace({'TrackOrigin':{'OriginalDriver':'Driver', 'Taxi':'Driver'}})

In [ ]:
TC.loc[:,'TrackOrigin'].unique()

array(['Driver', 'Passanger'], dtype=object)

In [20]:
#Prepare data
X = TC.loc[:,TC.columns != 'TrackOrigin']
y = TC.loc[:,'TrackOrigin']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [21]:
# DecisionTreeClassifier model
tree = DecisionTreeClassifier(criterion='entropy', max_depth=26, min_samples_leaf=4, splitter="random", random_state=1)

In [22]:
# DecisionTreeClassifier model train
tree.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=26, min_samples_leaf=4,
                       random_state=1, splitter='random')

Execution time 17s

In [23]:
# DecisionTreeClassifier model prediction
DTC_pred = tree.predict(X_test)

In [24]:
# Accuracy score DecisionTreeClassifier
print(accuracy_score(DTC_pred, y_test))

0.8397071172394309


Accuracy 0.8397071172394309

In [25]:
# BaggingClassifier with tree model train
model = BaggingClassifier(DecisionTreeClassifier(splitter='random'), n_estimators=34, random_state=1)
model.fit(X_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(splitter='random'),
                  n_estimators=34, random_state=1)

Execution time 5m (n_estimators=34)

In [26]:
# BaggingClassifier with tree model prediction
BC_pred = model.predict(X_test)

In [27]:
# Accuracy score BaggingClassifier with tree
print(accuracy_score(BC_pred, y_test))

0.9105786762724678


Accuracy 0.9105786762724678(n_estimators=34)

In [ ]:
# Search parametrs RandomForestClassifier with RandomizedSearchCV
n_estimators = [int(x) for x in np.linspace(start = 30, stop = 50, num = 10)]
min_samples_split = [int(x) for x in np.linspace(start = 30, stop = 50, num = 10)]
min_samples_leaf = [int(x) for x in np.linspace(start = 10, stop = 30, num = 10)]

param_dist = {'n_estimators': n_estimators,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

rs = RandomizedSearchCV(RandomForestClassifier(), 
                        param_dist, 
                        n_iter = 4, 
                        cv = 2, 
                        verbose = 1, 
                        n_jobs=-1, 
                        random_state=0)
rs.fit(X_train, y_train)
rs.best_params_

Fitting 2 folds for each of 4 candidates, totalling 8 fits


{'min_samples_leaf': 12, 'min_samples_split': 50, 'n_estimators': 34}

In [ ]:
# RandomForestClassifier model prediction
RFCs_pred = rs.predict(X_test)

In [ ]:
# Accuracy score RandomForestClassifier
print(accuracy_score(RFCs_pred, y_test))

0.8376392017139763


In [ ]:
# RandomForestClassifier model train
forest = RandomForestClassifier(n_estimators=34, n_jobs=-1, random_state=1)
forest.fit(X_train, y_train)

RandomForestClassifier(n_estimators=34, n_jobs=-1, random_state=1)

Execution time 7m

In [ ]:
# RandomForestClassifier model prediction
RFC_pred = forest.predict(X_test)

In [ ]:
# Accuracy score RandomForestClassifier
print(accuracy_score(RFC_pred, y_test))

0.8466917275531105


Accuracy 0.8466917275531105

In [ ]:
# ExtraTreesClassifier model train
Eforest = ExtraTreesClassifier(n_estimators=34, n_jobs=-1, random_state=1)
Eforest.fit(X_train, y_train)

Execution time 3m

In [ ]:
# ExtraTreesClassifier model prediction
EFC_pred = Eforest.predict(X_test)

In [ ]:
# Accuracy score ExtraTreesClassifier
print(accuracy_score(EFC_pred, y_test))

0.824581642769882
